## Step # 01: Install necessary libraries

In [ ]:
%pip install 'crewai[tools]'==0.28.8      
%pip install langchain-groq==0.1.3        
%pip install duckduckgo-search==5.3.0 
%pip install pandas

## Step # 02: Import necessary libraries

In [1]:
import os                                         #for interacting with operating system(file paths, env-variables)
import re                                         #for working with regular expressions(text pattern matching)                                     
from datetime import datetime                     #for date and time

import pandas as pd                               #for data handling
import requests                                   #for web requests
from crewai import Agent, Crew, Process, Task     #framework to build autonomous AI Agents
from crewai_tools import tool
from langchain.agents import load_tools           #LangChain is a framework used to build applications powered by large language models (LLMs)
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


/home/mahrukh/NLP/agents-with-llama-3/venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:898: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [2]:
#Function: to process a string containing multiple comma-separated entries enclosed in square brackets and return a cleaned list of those entries.
def format_response(response:str ) -> str:
    entries = re.split(r"(?<=]),(?=\[])", response)
    return [entry.strip("[]") for entry in entries]

In [ ]:
os.environ["GROQ_API_KEY"] = "<YOUR_GROQ_API_KEY_HERE>"

## Step # 03: Initializes a DuckDuckGo Search tool

In [4]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [ ]:
response = search_tool.run("Earthquake")
format_response(response)

In [ ]:
response = search_tool.run("Rainfall")
format_response(response)

In [7]:
human_tools = load_tools(["human"])

In [8]:
human_tools = load_tools(["human"])

## Step # 04: Create Dataframe of Fetched Earthquake Data

In [10]:
# Fetch recent earthquake data:
def get_recent_eathquake_data() -> pd.DataFrame:
    url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson"
    response = requests.get(url)
    data = response.json()
    features = data["features"]

    records = [
        {
            "place": feature["properties"]["place"],
            "magnitude": feature["properties"]["mag"],
            "time": datetime.utcfromtimestamp(feature["properties"]["time"] / 1000),
            "url": feature["properties"]["url"]
        } for feature in features
    ]

    df = pd.DataFrame(records)

    # Correctly cut the magnitudes into severity categories
    df["severity"] = pd.cut(
        df["magnitude"],
        bins=[0, 2, 4, 6, 7.5, 10],
        labels=["Very Minor", "Minor", "Moderate", "Strong", "Major"]
    )

    return df


In [11]:
quake_df = get_recent_eathquake_data()
quake_df.head()

,place,magnitude,time,url,severity
0,"14 km SE of Anza, CA",0.84,2025-04-17 08:41:41.330,https://earthquake.usgs.gov/earthquakes/eventp...,Very Minor
1,"7 km NW of The Geysers, CA",0.75,2025-04-17 08:37:02.240,https://earthquake.usgs.gov/earthquakes/eventp...,Very Minor
2,"12 km NW of Anza, CA",0.79,2025-04-17 08:36:05.250,https://earthquake.usgs.gov/earthquakes/eventp...,Very Minor
3,"9 km ESE of Alum Rock, CA",1.56,2025-04-17 08:23:08.900,https://earthquake.usgs.gov/earthquakes/eventp...,Very Minor
4,"40 km NW of Ninilchik, Alaska",2.50,2025-04-17 08:09:48.107,https://earthquake.usgs.gov/earthquakes/eventp...,Minor


## Step # 05: Define Tools

In [13]:
#Format the first 10 Dataframe and prints multi-line string
text_output = []
for index, row in quake_df.head(15).iterrows():
    text_output.append(f"{row['time'].strftime('%Y-%m-%d %H:%M:%S')} | {row['place']} | Magnitude: {row['magnitude']:.1f} | Severity: {row['severity']}") #fromats magnitude to 1 decimal place
formatted_text="\n".join(text_output)
print(formatted_text)

2025-04-17 08:41:41 | 14 km SE of Anza, CA | Magnitude: 0.8 | Severity: Very Minor
2025-04-17 08:37:02 | 7 km NW of The Geysers, CA | Magnitude: 0.8 | Severity: Very Minor
2025-04-17 08:36:05 | 12 km NW of Anza, CA | Magnitude: 0.8 | Severity: Very Minor
2025-04-17 08:23:08 | 9 km ESE of Alum Rock, CA | Magnitude: 1.6 | Severity: Very Minor
2025-04-17 08:09:48 | 40 km NW of Ninilchik, Alaska | Magnitude: 2.5 | Severity: Minor
2025-04-17 08:09:32 | 16 km N of Ocotillo Wells, CA | Magnitude: 1.0 | Severity: Very Minor
2025-04-17 08:06:24 | 2 km E of The Geysers, CA | Magnitude: 0.9 | Severity: Very Minor
2025-04-17 08:03:25 | 2 km WSW of Julian, CA | Magnitude: 0.7 | Severity: Very Minor
2025-04-17 08:02:20 | 8 km NNW of The Geysers, CA | Magnitude: 0.6 | Severity: Very Minor
2025-04-17 07:50:16 | 5 km NE of Loyalton, California | Magnitude: 1.5 | Severity: Very Minor
2025-04-17 07:36:44 | 4 km SW of Indios, Puerto Rico | Magnitude: 2.2 | Severity: Minor
2025-04-17 07:29:16 | 16 km WSW o

In [ ]:
#Create a Earthquake tool
@tool("earthquake tool")
def recent_earthquake_tool() -> str:
    """Get recent earthquake data for the past day (top 60 events)"""
    quake_df = get_recent_eathquake_data()
    text_output = []
    for index, row in quake_df.head(60).iterrows():
        text_output.append(
            f"{row['time'].strftime('%Y-%m-%d %H:%M:%S')} | {row['place']} | Magnitude: {row['magnitude']:.1f} | Severity: {row['severity']}"
        )
    return "\n".join(text_output)


In [20]:
@tool("search tool")
def earthquake_news_tool(location: str ) -> str:
    """Get recent news realted to earthquakes in a given location"""
    return search_tool.run(location+"earthquake news")

## Step # 06: Using Llama-3 Model with Groq

In [ ]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192") #temperature controls how random AI's output will be -> temp=0 (makes it deterministic)

In [26]:
%%time
system="You are an experienced Machine Learning & AI-Engineer."
human= "{text}"
prompt= ChatPromptTemplate.from_messages([("system", system),("human",human)])
chain=prompt | llm    #pipe operator (|) to compose a prompt with a language model (LLM), effectively creating a processing chain.
response = chain.invoke({"text":"What are the difference techniques of Fine-Tuning?"})
print(response.content)

Fine-tuning is a powerful technique in deep learning that involves adjusting the pre-trained model's weights to fit a new task or dataset. There are several techniques to fine-tune a pre-trained model, and I'll outline some of the most popular ones:

1. **Last Layer Fine-Tuning**: This involves freezing all the layers except the last layer (usually the classification layer) and updating only the last layer's weights to fit the new task. This technique is useful when the pre-trained model's features are general enough to be applicable to the new task.

2. **Full Model Fine-Tuning**: In this technique, all the layers of the pre-trained model are updated during fine-tuning. This approach is useful when the new task requires significant changes to the model's features.

3. **Freeze and Unfreeze**: This technique involves freezing certain layers of the pre-trained model and unfreezing others. The frozen layers are not updated during fine-tuning, while the unfrozen layers are updated. This a

## Step # 07: Initialize Agents

In [29]:
#Senior Environmental Customer Communicator
customer_communicator = Agent (
    role="Senior environmental and natural events communicator",
    goal="Understand what natural events the customer is asking about, such as earthquake, rainfall, weather patterns, or environmental changes, and provide clear, accurate, and relevanr information",
    backstory="""You're a highly experienced specialist in communicating about natural events including earthquakes, rainfall, weather, and environmental conditions. 
    You help users by answering their questions and providing helpful explanations, facts and insights on these topics.""",
    verbose=True, 
    allow_delegation=False,     #handles things directly without delegating
    llm=llm,
    max_iter=5,
    memory=True,                #whether this agent remembers context/history
    tools=human_tools,          #tools available to this agent(search tools, APIs etc)
    )

#Enviornmental News Analyst Agent
news_analyst = Agent(
    role="Earthquake News Analyst",
    goal="""Get recent earthquake news. Write 1 paragraph analysis of recent seismic activities 
    and classify the trend — increasing, decreasing, or stable.""",
    backstory="""You're an expert analyst of natural disaster trends based on earthquake news. 
    You have a thorough understanding of seismological patterns and global tectonic activities, 
    specializing in analyzing the news impact.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[earthquake_news_tool],
)

#Environmental Newsletter Writer
writer = Agent (
    role="Environmental Newsletter Writer",
    goal="""Write an insightful, compelling, and informative 3-paragraph-long newsletter based on the latest environmental events and natural phenomena report.""",
    backstory="""You're deeply familiar with natural events, environmental science, and climate-related topics. 
    You understand complex concepts like earthquake science, rainfall patterns, climate trends, and their real-world impacts — and have a gift for transforming this technical information into engaging, clear, and easy-to-read articles for a general audience.

    You're a master of communication, using narratives and structure to keep readers engaged.
    Your writing focuses on clarity, conciseness, and a pleasant flow that makes technical content approachable.

    Some of your writing techniques include:

    - Start with a 3-bullet executive summary highlighting the most important points
    - Use bullet points and subheadings to make content skimmable and easy to follow
    - Use real-world examples and analogies to explain complex environmental ideas
    - Craft compelling introductions and conclusions to make your writing memorable and actionable

    Your writing style turns even the driest scientific data into engaging and interesting newsletters for readers of all backgrounds. """,
    verbose=True,
    allow_delegation=True,    # can delegate tasks to other agents if needed
    llm=llm,
    max_iter=10,
    memory=True,)

## Step # 08: Initialize Tasks for Agents 

In [30]:
#Task for Senior Environmental Customer Communicator Agent
get_natural_event_topics = Task (
    description=f"""Ask the user which natural events or environmental topics they're interested in (like earthquake, rainfall, temperature trends, storms, or other climate events).""",
    expected_output="""Create a list of the natural events or environmental topics that the user wants information about.
    Example: 
    events = 'earthquake, rainfall, storm'
    """,
    agent=customer_communicator,
)

#Task for Environmental Events Analyst Agent
get_event_news_and_data = Task(
    description = f"""Take the list of natural events and always include 'earthquake' and 'rainfall' to it(if not requested)
    Use the search tool to gather news, updates, and scientific data for each topic individually. 
    The current date is {datetime.now()}.
    Summarize the information into a clear, helpful report.""",
    expected_output="""A summary of the current environmental situation and a one-sentence summary for
    each requested natural event or topic. Include a risk or impact score between 0 and 100 (0 being no risk/impact, 100 being severe risk/impact). Use this format:
    <EVENT>
    <SUMMARY BASED ON NEWS AND DATA>
    <CURRENT TREND OR PATTERN>
    <RISK/IMPACT SCORE>
    """,
    agent=news_analyst,
    context=[get_natural_event_topics],
)

#Task for Environmental Newsletter Writer Agent
write_environment_newsletter = Task(
    description = f"""
    Use the environmental events report to create a daily newsletter that highlights the most important points.
    Focus on the event summaries, current trends, and risk/impact scores. Discuss near-term and future considerations.
    The newsletter should be accessible to a wide audience and rely on clear, factual, and evidence-based information.""",
    expected_output = """An eloquent 3-paragraph newsletter formatted as markdown in an easy-to-read manner. It should contain: 
    - 3-bullet executive summary 
    - Introduction — set the overall context and hook the reader's interest
    - Main section — detailed analysis including news summaries and risk/impact scores
    - Summary — key facts and a future prediction: increasing, descreasing, or stable trend               
    """,
    agent=writer,
    context=[get_event_news_and_data],    
)

## Step # 09: Initialize Crew

In [ ]:
crew = Crew(
    agents=[customer_communicator, news_analyst, writer],
    tasks=[get_natural_event_topics,get_event_news_and_data, write_environment_newsletter],
    verbose=0,   #Detailed logs
    Process=Process.sequential, #how to coordinate these tasks
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15,
)

2025-04-17 15:16:15,477 - 137977149190144 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
results=crew.kickoff()     #kickoff() -> method that triggers the execution